## Oil splill

### Objective

Apply the oil spill detection

### The Python modules required

In [94]:
from string import Template
from shapely.geometry import box
from shapely.wkt import loads

import os
import gdal, osr
import dateutil.parser as parser

from timeit import default_timer as timer


import sys 

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

import cioppy

from geopandas import GeoDataFrame
import pandas as pd

from ipyleaflet import *



import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import HTML


import shapely
from shapely.geometry import Point
from shapely.geometry import LineString

import snappy

sys.path.append(os.getcwd())
import ellip_snap_helpers


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
point_lat, point_lon = 70.704004,-55.8380066

In [61]:
toi = loads('POINT(%s %s)' % (point_lon, point_lat))

In [62]:
start_time = '2018-08-20T00:00:00Z'
stop_time = '2018-08-26T23:59:59Z'

In [63]:
search_params = dict([('geom', toi.wkt),
                      ('start', start_time),
                      ('stop', stop_time),
                      ('do', 'terradue'),
                      ('pt', 'GRD')])

In [64]:
ciop = cioppy.Cioppy()

series = 'https://catalog.terradue.com/sentinel1/search'

s1_results = GeoDataFrame(ciop.search(end_point=series, 
                     params=search_params,
                     output_fields='identifier,self,startdate,track,wkt,enclosure',
                     model='EOP'))

In [75]:
s1_results['wkt'] = s1_results['wkt'].apply(shapely.wkt.loads)

In [76]:
s1_results

,enclosure,identifier,self,startdate,track,wkt
0,https://store.terradue.com/download/sentinel1/...,S1B_EW_GRDH_1SDH_20180826T205512_20180826T2056...,https://catalog.terradue.com/sentinel1/search?...,2018-08-26T20:55:12.8795820Z,163,"POLYGON ((-64.169556 72.710106, -51.521824 73...."
1,https://store.terradue.com/download/sentinel1/...,S1B_EW_GRDM_1SDH_20180826T205512_20180826T2056...,https://catalog.terradue.com/sentinel1/search?...,2018-08-26T20:55:12.8833320Z,163,"POLYGON ((-64.170158 72.70983099999999, -51.52..."
2,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SDH_20180826T102423_20180826T1025...,https://catalog.terradue.com/sentinel1/search?...,2018-08-26T10:24:23.2927260Z,69,"POLYGON ((-51.140747 67.82652299999999, -60.96..."
3,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDH_1SDH_20180826T102423_20180826T1025...,https://catalog.terradue.com/sentinel1/search?...,2018-08-26T10:24:23.2970000Z,69,"POLYGON ((-51.141376 67.826599, -60.961288 68...."
4,https://store.terradue.com/download/sentinel1/...,S1B_EW_GRDM_1SDH_20180825T103202_20180825T1033...,https://catalog.terradue.com/sentinel1/search?...,2018-08-25T10:32:02.5420000Z,142,"POLYGON ((-53.421989 67.65634900000001, -63.10..."
5,https://store.terradue.com/download/sentinel1/...,S1B_EW_GRDH_1SDH_20180825T103202_20180825T1033...,https://catalog.terradue.com/sentinel1/search?...,2018-08-25T10:32:02.5380000Z,142,"POLYGON ((-53.422573 67.656471, -63.102581 68...."
6,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SDH_20180824T104049_20180824T1041...,https://catalog.terradue.com/sentinel1/search?...,2018-08-24T10:40:49.5280000Z,40,"POLYGON ((-54.939865 68.12882999999999, -64.94..."
7,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDH_1SDH_20180824T104049_20180824T1041...,https://catalog.terradue.com/sentinel1/search?...,2018-08-24T10:40:49.5280000Z,40,"POLYGON ((-54.940437 68.128998, -64.945824 69...."
8,https://store.terradue.com/download/sentinel1/...,S1B_EW_GRDM_1SDH_20180823T104821_20180823T1049...,https://catalog.terradue.com/sentinel1/search?...,2018-08-23T10:48:21.5310000Z,113,"POLYGON ((-57.093029 68.135239, -67.0763550000..."
9,https://store.terradue.com/download/sentinel1/...,S1B_EW_GRDH_1SDH_20180823T104821_20180823T1049...,https://catalog.terradue.com/sentinel1/search?...,2018-08-23T10:48:21.5290000Z,113,"POLYGON ((-57.093506 68.13549, -67.07659099999..."


In [77]:
global m

from ipyleaflet import Map, Polygon

m = Map(center=(toi.y, 
                toi.x), zoom=5)


marker = Marker(location=(toi.y, 
                toi.x), draggable=False)

m.add_layer(marker);

m


TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [78]:
global layer_group 

layer_group = LayerGroup(layers=())
m.add_layer(layer_group)

def f(x):
    layer_group.clear_layers()
    m.remove_layer(layer_group)
        
    p = Polygon(locations=np.asarray([t[::-1] for t in list(list(s1_results.iloc[[x]]['wkt'])[0].exterior.coords)]).tolist(), color="red", fill_color="green")
    
    d = {'identifier': list(s1_results.iloc[[x]]['identifier'])[0], 
         'track' :list(s1_results.iloc[[x]]['track'])[0]}
    
    html_value="""
        <div>
        <ul class='list-group'>
            <li class='list-group-item'>{identifier}</li>
            <li class='list-group-item'>{track}</li>
        </ul></div>""".format(**d)
    
    
    html_widget_slave = HTML(
            value=html_value,
    placeholder='',
    description='',
    )
    
    
    
    layer_group.add_layer(p)
    p.popup = html_widget_slave
    m.add_layer(layer_group)

In [79]:
interact(f, x=widgets.IntSlider(min=0,max=len(s1_results)-1,step=1,value=0));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J3gnLCBtYXg9MTEpLCBPdXRwdXQoKSksIF9kb21fY2xhc3Nlcz0odSd3aWRnZXQtaW50ZXJhY3TigKY=


In [80]:
s1_results.iloc[2]

enclosure     https://store.terradue.com/download/sentinel1/...
identifier    S1A_EW_GRDM_1SDH_20180826T102423_20180826T1025...
self          https://catalog.terradue.com/sentinel1/search?...
startdate                          2018-08-26T10:24:23.2927260Z
track                                                        69
wkt           POLYGON ((-51.140747 67.82652299999999, -60.96...
Name: 2, dtype: object

In [82]:
import os

target_dir = '/workspace/data2'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

ciop.copy(urls=s1_results.iloc[2]['enclosure'], extract=False, target=target_dir)

'/workspace/data2/S1A_EW_GRDM_1SDH_20180826T102423_20180826T102527_023416_028C53_A49B.zip'

In [95]:
ellip_snap_helpers.get_operator_help('Land-Sea-Mask')

Operator name: org.esa.snap.raster.gpf.masks.CreateLandMaskOp
Operator alias: Land-Sea-Mask

Parameters:

sourceBandNames: The list of source bands.
Default Value: None

Possible values: []

landMask: None
Default Value: true

Possible values: []

useSRTM: None
Default Value: true

Possible values: []

geometry: None
Default Value: None

Possible values: []

invertGeometry: None
Default Value: false

Possible values: []

shorelineExtension: None
Default Value: 0

Possible values: []



In [106]:
parameters = ellip_snap_helpers.get_operator_default_parameters('Oil-Spill-Clustering')

In [107]:
parameters

{'minClusterSizeInKm2': '0.1'}

In [100]:
parameters['backgroundWindowSize'] = '61'


In [101]:
parameters

{'geometry': None,
 'invertGeometry': 'false',
 'landMask': 'true',
 'shorelineExtension': '10',
 'sourceBandNames': None,
 'useSRTM': 'false'}

In [67]:
def oil_spill(row, aoi):
    
    local_path = row['local_path']
    identifier = row['identifier']
    
    mygraph = ellip_snap_helpers.GraphProcessor()
    
    operator = 'Read'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    parameters['file'] = local_path 
    mygraph.add_node(operator,
                     operator, 
                     parameters,
                     '') 
    
    operator = 'Land-Sea-Mask'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    parameters['useSRTM'] = 'false'
    parameters['shorelineExtension'] = '10'
    
    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'Read')
    

    operator = 'Apply-Orbit-File'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator, 
                     operator, 
                     parameters, 
                     'Land-Sea-Mask')



    operator = 'Calibration'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator,
                     operator,
                     parameters, 
                     'Apply-Orbit-File')


    operator = 'Oil-Spill-Detection'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    parameters['backgroundWindowSize'] = '61'
    
    mygraph.add_node(operator, 
                     operator,
                     parameters, 
                     'Calibration')


    operator = 'Oil-Spill-Clustering'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'Oil-Spill-Detection')


    


    operator = 'Write'
    
    output_name = 'oil_%s' % identifier
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    parameters['file'] = output_name
    parameters['formatName'] = 'GeoTIFF-BigTiff'
    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'Subset')

    mygraph.run()
    
    row['oil_spill'] = output_name + '.tif'
    
    return row

In [110]:
selected = GeoDataFrame(s1_results.iloc[2]).T

In [111]:
selected

,enclosure,identifier,self,startdate,track,wkt
2,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SDH_20180826T102423_20180826T1025...,https://catalog.terradue.com/sentinel1/search?...,2018-08-26T10:24:23.2927260Z,69,"POLYGON ((-51.140747 67.82652299999999, -60.96..."


In [112]:
target_dir = '/workspace/data2'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

def stage_in(row):
    
    local_path = ciop.copy(row['enclosure'], extract=False, target=target_dir)
    row['local_path'] = local_path
        
    return row 

In [113]:
selected = selected.apply(lambda row: stage_in(row), axis=1)

In [114]:
selected

,enclosure,identifier,self,startdate,track,wkt,local_path
2,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SDH_20180826T102423_20180826T1025...,https://catalog.terradue.com/sentinel1/search?...,2018-08-26T10:24:23.2927260Z,69,"POLYGON ((-51.140747 67.82652299999999, -60.96...",/workspace/data2/S1A_EW_GRDM_1SDH_20180826T102...


In [117]:
extended_aoi = None

In [118]:
selected = selected.apply(lambda row: oil_spill(row, extended_aoi), axis=1)

Processing the graph
Process PID: 26165
Executing processing graph
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.snap.engine_utilities.download.downloadablecontent.DownloadableContentImpl: http retrieving http://step.esa.int/auxdata/orbits/Sentinel-1/POEORB/S1A/2018/08/S1A_OPER_AUX_POEORB_OPOD_20180915T120754_V20180825T225942_20180827T005942.EOF.zip
INFO: org.esa.snap.core.dataop.dem.ElevationFile: http retrieving http://step.esa.int/auxdata/dem/ACE30/60N075W.zip
INFO: org.esa.snap.core.dataop.dem.ElevationFile: http retrieving http://step.esa.int/auxdata/dem/ACE30/60N060W.zip
INFO: org.esa.snap.core.dataop.dem.ElevationFile: http retrieving http://step.esa.int/auxdata/dem/ACE30/60N045W.zip

Done.
Processing the graph
Process PID: 26264
Executing processing graph
....10%....20%....30%....40%.

## License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.